In [1]:
import numpy as np
import pandas as pd
import h5py
import os
from scipy.stats import skew, kurtosis
from scipy import signal, stats
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
path = '/scratch/qh503/deepLearningProject/data_h5/'

In [3]:
data_dict = h5py.File(os.path.join(path, "raw_EEG.h5"), "r")
data = data_dict['data'][:]
tasks = data_dict['tasks'][:]
reps = data_dict['reps'][:]
trials = data_dict['trials'][:]
trial_reps = data_dict['trial_reps'][:]
subjects = data_dict['subjects'][:]

In [4]:
# train-valid-test split: # about 72-14-14
np.random.seed(42)
which_trial_reps = np.unique(trial_reps)
tr_trial_reps = np.random.choice(which_trial_reps, 5, replace=False)
val_trial_reps = 4
ts_trial_reps = 7

tr_ind = np.where((trial_reps == 1) | (trial_reps == 2) | (trial_reps == 3) | (trial_reps == 5) | (trial_reps == 6))[0]
val_ind = np.where(trial_reps == val_trial_reps)[0]
ts_ind = np.where(trial_reps == ts_trial_reps)[0]

tr_data, tr_tasks = data[tr_ind], tasks[tr_ind]
val_data, val_tasks = data[val_ind], tasks[val_ind]
ts_data, ts_tasks = data[ts_ind], tasks[ts_ind]

In [5]:
# Xtr, ytr = tr_data, tr_tasks
# Xval, yval = val_data, val_tasks
# Xts, yts = ts_data, ts_tasks

# test on small amount of data
Xtr, ytr = tr_data[np.where(subjects[tr_ind] <= 15)[0]], tr_tasks[np.where(subjects[tr_ind] <= 15)[0]]
Xval, yval = val_data[np.where(subjects[val_ind] <= 15)[0]], val_tasks[np.where(subjects[val_ind] <= 15)[0]]
Xts, yts = ts_data[np.where(subjects[ts_ind] <= 15)[0]], ts_tasks[np.where(subjects[ts_ind] <= 15)[0]]
tr_subjects = subjects[tr_ind][np.where(subjects[tr_ind] <= 15)[0]]
val_subjects = subjects[val_ind][np.where(subjects[val_ind] <= 15)[0]]
ts_subjects = subjects[ts_ind][np.where(subjects[ts_ind] <= 15)[0]]

In [6]:
print(Xtr.shape, ' ', ytr.shape)
print(Xval.shape, ' ', yval.shape)
print(Xts.shape, ' ', yts.shape)

(1800, 640, 64)   (1800,)
(360, 640, 64)   (360,)
(360, 640, 64)   (360,)


In [7]:
f_train = h5py.File(os.path.join(path, 'train1800_raw_EEG.h5'), 'w')
f_train.create_dataset('data', data=Xtr)
f_train.create_dataset('tasks', data=ytr)
f_train.create_dataset('subjects', data=tr_subjects)

f_valid = h5py.File(os.path.join(path, 'valid360_raw_EEG.h5'), 'w')
f_valid.create_dataset('data', data=Xval)
f_valid.create_dataset('tasks', data=yval)
f_valid.create_dataset('subjects', data=val_subjects)

f_test = h5py.File(os.path.join(path, 'test360_raw_EEG.h5'), 'w')
f_test.create_dataset('data', data=Xts)
f_test.create_dataset('tasks', data=yts)
f_test.create_dataset('subjects', data=ts_subjects)

f_train.close()
f_valid.close()
f_test.close()